In [3]:
from IPython.display import display, clear_output
from ipywidgets import interact, interactive,fixed, IntSlider, FloatSlider, HBox, Layout, Output, VBox, HTML,HTMLMath,Box,Text,link,FloatText
import numpy as np 
import matplotlib.pyplot as plt
from scipy import integrate
from numpy import *

HTML("""
<style>
.container {
  position: relative;
  text-align: center;
  color: white;
}

.bottom-left {
  position: absolute;
  bottom: 8px;
  left: 16px;
}

.top-left {
  position: absolute;
  top: 8px;
  left: 16px;
}

.top-right {
  position: absolute;
  top: 8px;
  right: 16px;
}

.bottom-right {
  position: absolute;
  bottom: 8px;
  right: 16px;
}

.centered {
  position: absolute;
  top: 50%;
  left: 50%;
  transform: translate(-50%, -50%);
}

</style>
    """)


HTML(value='\n<style>\n.container {\n  position: relative;\n  text-align: center;\n  color: white;\n}\n\n.bott…

In [4]:
%matplotlib inline

out = Output(layout={'width': '60%'})

N = 1000000  # initial population size
I = 10       # initial infected
R = 0        # initial removed

def update_plot(a,b): 
    
    with out: 
        clear_output(wait=True) # prevents distortions
        
        fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize=(16, 8))

        def dX_dt(X, t=0):
            """ Return the susceptible and infected population. """
            return array([ -   a*X[0]*X[1]/N ,              # susceptible
                               a*X[0]*X[1]/N - b*X[1],      # infected
                                               b*X[1]])     # removed
        #t = linspace(0, 90,  180)              # time
        t=np.arange(0,90,0.5)
        X0 = array([N, I,R])                     # initials conditions: 10 susceptible and 0 infected
        X, infodict = integrate.odeint(dX_dt, X0, t, full_output=True)
        susceptible, infected,removed = X.T        

        scat, = ax1.plot(t, susceptible,'r-',label='susceptible',linewidth=4)
        scat, = ax1.plot(t, infected,'b-',label='infected',linewidth=4)
        scat, = ax1.plot(t, removed,'k-',label='removed',linewidth=4)
        ax1.legend(loc='best')
        ax1.grid()
        ax1.set_title('susceptible and infected part of the population',fontsize=18)
        ax1.set_xlabel('time [days]', fontsize=15)
        ax1.set_ylabel('population', fontsize=15)
        ax1.set_title('susceptible and infected',fontsize=18)

        # we are on this line:
        # V = delta*x -gamma*ln(x) + beta*y - alpha*ln(y)
        #our_V = c*X0[0] -d*log(X0[0]) + b*X0[1] - a*log(X0[1])
        # ---------------------------------------------------------
        values  = linspace(0.33, 2, 6)                          # position of the orbits around our initial conditions
        
                
        vcolors = plt.cm.autumn_r(linspace(0.3, 1., len(values)))  # colors for each trajectory
        #X_f0 = array([     0. ,  0.])
        #X_f1 = array([ d/c, a/b])  # stable point (equilibrium)
        #equil = "equilibrium point = (" + str(X_f1[0]) + "," + str(X_f1[1]) + ")"
    
        #------------------------------------------------------------------
        # plot trajectories
        for v, col in zip(values, vcolors):
        
            #X0 = v * X_f1                               # starting point
            X_0 = v*X0                               # starting point
            X = integrate.odeint( dX_dt, X_0, t)         # we don't need infodict here
            ax2.plot( X[:,0], X[:,1], lw=5.0, color=col, label='X0=(%.f, %.f)' % ( X_0[0], X_0[1]) )

        # location of the equilibrium point     
        #ax2.plot(X_f1[0],X_f1[1],'o')    
        
        #-------------------------------------------------------
        # define a grid and compute direction at each point
        ymax = plt.ylim(ymin=0)[1]                        # get axis limits
        xmax = plt.xlim(xmin=0)[1]
        nb_points   = 20

        x = linspace(0, xmax, nb_points)
        y = linspace(0, ymax, nb_points)

        X1 , Y1  = meshgrid(x, y)                       # create a grid
        DX1, DY1, DZ1 = dX_dt([X1, Y1])                 # compute growth rate on the grid (for S,I,R)
        M = (hypot(DX1, DY1))                           # Norm of the growth rate 
        M[ M == 0] = 1.                                 # Avoid zero division errors 
        DX1 /= M                                        # Normalize each arrows
        DY1 /= M

        ax2.set_title('Trajectories and direction fields',fontsize=18)
        ax2.set_xlabel('Number of Susceptible', fontsize=15)
        ax2.set_ylabel('Number of Infected',fontsize=15)
        Q=ax2.quiver(X1, Y1, DX1, DY1, M, pivot='mid', cmap=plt.cm.jet)
    


        
        #fig.tight_layout()
        plt.show()
        fig.savefig('susceptible_and_infected.png')

        
        
headertextbox = HTML(value="""
<div class="container">
<img src="https://raw.githubusercontent.com/bigfooted/covid19-seir/master/coronavirus_cropped.jpg" alt="corona" style="width:100%;margin:-8px 0px">
   <div class="centered">
          <font size="10">
          SIR model of a virus outbreak
              </font> 
     </div>
</div>
    """)




#textbox_R0 = Text(
#value=R0text
#    ,layout=Layout(width='200px')
#)



textbox = HTMLMath(
value="""<body><font size="4"> Classic SIR model: Kermack-McKendrick: <br> \
        $$\\dot S = -\\beta I S/N$$<br> \
        $$\\dot I = +\\beta I S/N - \\gamma I $$ <br> \
        $$\\dot R = \\gamma I$$ <br> \
        $S(t)$ : number of Susceptible persons (persons that are not infected or immune) <br> \
        $I(t)$ : number of Infected persons <br> \
        $R(t)$ : number of Removed persons (immune or dead) <br> \
        $N = S+I+R$ : total population <br> \
        $\\beta=1/T_l$ : infection rate = inverse of latent period <br> \
        $\\gamma=1/T_r$ : recovery rate = inverse of recovery time.  <br> \
        A virus can spread when the number of infections increases: $\\dot I>0$. This means that S=N, <br><br>\
        $$R_0 = \\beta / \\gamma > 1$$ \
        </font></body> 
       """
    ,layout=Layout(width='30%')
)


# to do: an output box displaying the location of the equilibrium plot


layout=Layout(border='0px solid black', width='400px',height='50px')

style = {'description_width': '150px','width':'500px'}
sliderInfectionrate = FloatSlider(min=0.2, max=2.0, step=0.1, value=0.4, description="infection rate:",orientation='horizontal',style=style,layout=layout) 
sliderRecoveryrate = FloatSlider(min=0.02, max=0.5, step=0.01, value=0.2, description="recovery rate:",orientation='horizontal',style=style,layout=layout) 
#l = link((sliderInfectionrate, 'value'), (textbox_R0, 'value'))

r0 = Output(
    layout={'width': '90%'}
)
def update_R0(a,b):
    with r0:
        clear_output(wait=True)
        # update the text
        print("      Basic reproduction number R0 = {a:2.2f}".format(a=(sliderInfectionrate.value/sliderRecoveryrate.value)))
        


sliders = interactive(update_plot,a=sliderInfectionrate,b=sliderRecoveryrate)
textr0 = interactive(update_R0,a=sliderInfectionrate,b=sliderRecoveryrate)

update_plot(a=0.4,b=0.2) # to show the initial plot
update_R0(a=0.4,b=0.2)   # to show the initial R0
display(VBox([headertextbox,HBox([out, VBox([sliderInfectionrate,sliderRecoveryrate, r0],layout=Layout(width='30%')), textbox])]))
